In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from fake_useragent import UserAgent
from time import sleep
import re

In [2]:
ua = UserAgent()
header = {'User-Agent':str(ua.chrome)}

url = "https://www.value.today/headquarters/united-states-america-usa?page=0"

source = requests.get(url, headers=header)
soup = BeautifulSoup(source.content, "html.parser")

In [3]:
info_blocks = soup.find_all(class_="node node--type-listed-companies node--view-mode-teaser ds-2col-stacked clearfix")

In [201]:
collected_names = []
collected_businesses = []
collected_values = []
collected_values_metrics = []
collected_revenues = []
collected_revenues_metrics = []
collected_op_incomes = []
collected_op_incomes_metrics = []
collected_net_incomes = []
collected_net_incomes_metrics = []
collected_assets = []
collected_assets_metrics = []
collected_liabilities = []
collected_liabilities_metrics = []

for item in info_blocks:

    name = item.find("h2").text.strip()
    collected_names.append(name)

    business = item.select("div.field--name-field-company-category-primary > div > div.field--item > a")[0].text.strip()
    collected_businesses.append(business)

    value = item.select("div.field--name-field-market-value-jan012021 > div.field--item")[0]["content"]
    collected_values.append(value)

    value_string = item.select("div.field--name-field-market-value-jan012021 > div.field--item")[0].text.strip()
    if "Billion" in value_string:
        collected_values_metrics.append("Billion")
    elif "Million" in value_string:
        collected_values_metrics.append("Million")

    revenue_string = item.select("div.field--name-field-annual-revenue > div.field--item")[0].text.strip()
    revenue = re.findall("\d+[.,]\d+", revenue_string)
    collected_revenues.append(revenue[0])

    if "Billion" in revenue_string:
        collected_revenues_metrics.append("Billion")
    elif "Million" in revenue_string:
        collected_revenues_metrics.append("Million")
        
    op_income_item = item.select("div.field--name-field-annual-operating-income > div.field--item")
    if len(op_income_item) == 0:
        collected_op_incomes.append(None)
    else:
        op_income_string = op_income_item[0].text.strip()
        op_income = re.findall("\d+[.,]\d+", op_income_string)
        collected_op_incomes.append(op_income[0])

    if len(op_income_item) == 0:
        collected_op_incomes_metrics.append(None)
    elif "Billion" in op_income_string:
        collected_op_incomes_metrics.append("Billion")
    elif "Million" in op_income_string:
        collected_op_incomes_metrics.append("Million")

    net_income_string = item.select("div.field--name-field-annual-revenue > div.field--item")[0].text.strip()
    net_income = re.findall("\d+[.,]\d+", net_income_string)
    collected_net_incomes.append(net_income[0])

    if "Billion" in net_income_string:
        collected_net_incomes_metrics.append("Billion")
    elif "Million" in net_income_string:
        collected_net_incomes_metrics.append("Million")

    assets_string = item.select("div.field--name-field-total-assets > div.field--item")[0].text.strip()
    assets = re.findall("\d+[.,]\d+", assets_string)
    collected_assets.append(assets[0])

    if "Billion" in assets_string:
        collected_assets_metrics.append("Billion")
    elif "Million" in assets_string:
        collected_assets_metrics.append("Million")

    liabilities_string = item.select("div.field--name-field-total-liabilities- > div.field--item")[0].text.strip()
    liabilities = re.findall("\d+[.,]\d+", liabilities_string)
    collected_liabilities.append(liabilities[0])

    if "Billion" in liabilities_string:
        collected_liabilities_metrics.append("Billion")
    elif "Million" in liabilities_string:
        collected_liabilities_metrics.append("Million")

dataframe = pd.DataFrame({"Company Name": collected_names, "Company Business": collected_businesses, "Market Value": collected_values, "Value Metric": collected_values_metrics, "Annual Revenue": collected_revenues, "Revenue Metric": collected_revenues_metrics, "Operating Income": collected_op_incomes, "Op Income Metric": collected_op_incomes_metrics, "Net Income": collected_net_incomes, "Net Income Metric": collected_net_incomes_metrics, "Assets": collected_assets, "Assets Metric": collected_assets_metrics, "Liabilities": collected_liabilities, "Liabilities Metric": collected_liabilities_metrics})

In [202]:
dataframe

,Company Name,Company Business,Market Value,Value Metric,Annual Revenue,Revenue Metric,Operating Income,Op Income Metric,Net Income,Net Income Metric,Assets,Assets Metric,Liabilities,Liabilities Metric
0,APPLE,Technology,2256,Billion,274.51,Billion,66.28,Billion,274.51,Billion,"317,344",Million,"245,062",Million
1,MICROSOFT CORPORATION,Technology,1682,Billion,143.00,Billion,52.95,Billion,143.00,Billion,"301,311",Million,"183,007",Million
2,ALPHABET,Technology,1185,Billion,182.52,Billion,41.22,Billion,182.52,Billion,299.243,Billion,86.32,Billion
3,AMAZON.COM,eCommerce,1634,Billion,386.06,Billion,22.89,Billion,386.06,Billion,"110,908",Million,"184,586",Million
4,FACEBOOK,Communication Services,778.04,Billion,85.96,Billion,32.67,Billion,85.96,Billion,"139,691",Million,"29,244",Million
5,BERKSHIRE HATHAWAY,Financial Services,543.68,Billion,327.22,Billion,None,None,327.22,Billion,829.946,Billion,410.73,Billion
6,J P MORGAN CHASE & CO,Financial Services,387.33,Billion,122.90,Billion,None,None,122.90,Billion,"3,386",Billion,"3,106",Billion
7,VISA,Financial Services,482.17,Billion,21.84,Billion,14.09,Billion,21.84,Billion,"77,884",Million,"42,360",Million
8,JOHNSON & JOHNSON,Healthcare,414.31,Billion,82.58,Billion,19.73,Billion,82.58,Billion,"158,380",Million,"95,402",Million
9,WALMART,Retail,407.84,Billion,523.96,Billion,22.54,Billion,523.96,Billion,"237,382",Million,"162,072",Million


In [142]:
info_blocks[2]

<div class="node node--type-listed-companies node--view-mode-teaser ds-2col-stacked clearfix" data-history-node-id="44288">
<div class="clearfix group-header">
<div class="field field--name-node-title field--type-ds field--label-hidden field--item"><h2 class="text-primary">
<a href="/company/alphabet" hreflang="en">ALPHABET</a>
</h2>
</div>
<div class="clearfix col-sm-6 field field--name-field-world-rank-jan012021 field--type-integer field--label-above">
<div class="field--label">World Rank (Jan-01-2021)</div>
<div class="field--item">5</div>
</div>
<div class="clearfix col-sm-6 field field--name-field-market-value-jan012021 field--type-float field--label-above">
<div class="field--label">Market Value (Jan-01-2021)</div>
<div class="field--item" content="1185">1185.000 Billion USD</div>
</div>
<div class="clearfix col-sm-6 field field--name-field-world-rank-jan-2020 field--type-integer field--label-above">
<div class="field--label">World Rank (Jan-2020)</div>
<div class="field--item">4

In [124]:
a = [1]

In [126]:
a[0]

1